# PCMCI Analysis of Seascape dependence

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline
import rpy2

import sklearn
import tigramite
from tigramite import data_processing as pp
from tigramite import plotting as tp
from tigramite.pcmci import PCMCI
from tigramite.independence_tests import ParCorr, GPDC, CMIknn, CMIsymb

Could not import rpy package
Could not import r-package RCIT


In [2]:
file_dir=r'C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Data'
file_dir2=r'C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour'

In [6]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [7]:
species_info=pd.read_csv(file_dir+"/spec_info.txt",sep="\t")
species_info.head()
for i in range(1,1):
    print(i)
    species=species_info["Scientific name"][i]
    print(species)
#    %%capture cap --no-stderr
   # try:
    spec=species.replace(" ","_")
    print(spec)
    my_data=np.genfromtxt(file_dir+'/%s_PCMCI_prep.csv'%spec,delimiter=",")
    my_data=np.nan_to_num(my_data)
    print(np.isnan(my_data).any())
        #print(my_data)
        # Initialize dataframe object, specify time axis and variable names
    var_names = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10"]
    dataframe = pp.DataFrame(my_data, 
                                 datatime = np.arange(len(my_data)), 
                                 var_names=var_names)

        #define function
    cmi_knn=CMIknn(significance="shuffle_test",knn=0.1, shuffle_neighbors=5,transform="ranks")
    pcmci_cmi_knn=PCMCI(dataframe=dataframe, cond_ind_test=cmi_knn,verbosity=2)
        #parcorr = ParCorr(significance='shuffle_test',confidence="bootstrap")
        #pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=1)

        #pcmci.verbosity = 1
        #results = pcmci.run_pcmci(tau_max=3, pc_alpha=None)
    results =pcmci_cmi_knn.run_pcmci(tau_max=3,pc_alpha=None)

        #q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
        
    pcmci_cmi_knn.print_significant_links(p_matrix=results["p_matrix"],val_matrix=results["val_matrix"],alpha_level=0.01)
        #pcmci.print_significant_links(
         #   p_matrix = results['p_matrix'], 
          #  q_matrix = q_matrix,
           # val_matrix = results['val_matrix'],
            #conf_matrix=results["conf_matrix"],
            #alpha_level = 0.01)
            
        
   # except:
    #    pass

#    with open(file_dir2+"/Seascapes/%s_PCMCI.txt"%spec,"a") as f:
#        f.write(cap.stdout)
#        f.close()
#    clear_output()
#except:
     #   pass

,N,Species,Scientific name,LW-conversion,Lmin (cm),Lmax (cm),a,b
0,1,Dab,Limanda limanda,0.0068*(L)^3.14,2,40.0,0.0068,3.14
1,2,Whiting,Merlangius merlangus,0.0063*(L)^3.06,2,91.5,0.0063,3.06
2,3,Herring,Clupea harengus,0.0060*(L)^3.08,2,45.0,0.0060,3.08
3,4,Cod,Gadus morhua,0.0071*(L)^3.08,2,200.0,0.0071,3.08
4,5,Sprat,Sprattus sprattus,0.0056*(L)^3.09,2,16.0,0.0056,3.09


In [10]:
my_data=np.genfromtxt(file_dir+'/Gadus_Morhua_PCMCI_prep.csv',delimiter=",")
my_data=np.nan_to_num(my_data)
print(np.isnan(my_data).any())
        #print(my_data)
        # Initialize dataframe object, specify time axis and variable names
var_names = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10"]
dataframe = pp.DataFrame(my_data, 
                         datatime = np.arange(len(my_data)), 
                         var_names=var_names)

#define function
#parcorr = ParCorr(significance='shuffle_test',confidence="bootstrap")
#pcmci = PCMCI(dataframe=dataframe, cond_ind_test=parcorr, verbosity=1)
cmi_knn=CMIknn(significance="shuffle_test",knn=0.1, shuffle_neighbors=5,transform="ranks")
pcmci_cmi_knn=PCMCI(dataframe=dataframe, cond_ind_test=cmi_knn,verbosity=2)

#pcmci.verbosity = 1
#results = pcmci.run_pcmci(tau_max=3, pc_alpha=None)
results =pcmci_cmi_knn.run_pcmci(tau_max=3,pc_alpha=0.05)
pcmci_cmi_knn.print_significant_links(p_matrix=results["p_matrix"],val_matrix=results["val_matrix"],alpha_level=0.01)
#q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
#pcmci.print_significant_links(
#            p_matrix = results['p_matrix'], 
#            q_matrix = q_matrix,
#            val_matrix = results['val_matrix'],
#            conf_matrix=results["conf_matrix"],
#            alpha_level = 0.01)

False

##
## Running Tigramite PC algorithm
##

Parameters:
independence test = cmi_knn
tau_min = 1
tau_max = 3
pc_alpha = 0.05
max_conds_dim = None
max_combinations = 1



## Variable S1

Iterating through pc_alpha = [0.05]:

# pc_alpha = 0.05 (1/1):

Testing condition sets of dimension 0:

    Link (S1 -1) --> S1 (1/30):
    Combination 0:  --> pval = 0.01700 / val = 0.507
    No conditions of dimension 0 left.

    Link (S1 -2) --> S1 (2/30):
    Combination 0:  --> pval = 0.01800 / val = 0.481
    No conditions of dimension 0 left.

    Link (S1 -3) --> S1 (3/30):
    Combination 0:  --> pval = 0.12700 / val = 0.365
    Non-significance detected.

    Link (S2 -1) --> S1 (4/30):
    Combination 0:  --> pval = 0.01800 / val = 0.463
    No conditions of dimension 0 left.

    Link (S2 -2) --> S1 (5/30):
    Combination 0:  --> pval = 0.72800 / val = 0.095
    Non-significance detected.

    Link (S2 -3) --> S1 (6/30):
    Combination 0:  --> pval = 0.14000 / val = 0.325
    Non-signi

    Combination 0:  --> pval = 0.32200 / val = 0.173
    Non-significance detected.

    Link (S6 -2) --> S2 (17/30):
    Combination 0:  --> pval = 0.10300 / val = 0.273
    Non-significance detected.

    Link (S6 -3) --> S2 (18/30):
    Combination 0:  --> pval = 0.03200 / val = 0.383
    No conditions of dimension 0 left.

    Link (S7 -1) --> S2 (19/30):
    Combination 0:  --> pval = 0.04800 / val = 0.401
    No conditions of dimension 0 left.

    Link (S7 -2) --> S2 (20/30):
    Combination 0:  --> pval = 0.01400 / val = 0.451
    No conditions of dimension 0 left.

    Link (S7 -3) --> S2 (21/30):
    Combination 0:  --> pval = 0.39500 / val = 0.210
    Non-significance detected.

    Link (S8 -1) --> S2 (22/30):
    Combination 0:  --> pval = 0.50100 / val = 0.154
    Non-significance detected.

    Link (S8 -2) --> S2 (23/30):
    Combination 0:  --> pval = 0.17800 / val = 0.284
    Non-significance detected.

    Link (S8 -3) --> S2 (24/30):
    Combination 0:  --> pval = 0

    Combination 0:  --> pval = 0.05000 / val = 0.347
    No conditions of dimension 0 left.

    Link (S8 -1) --> S3 (22/30):
    Combination 0:  --> pval = 0.33800 / val = 0.178
    Non-significance detected.

    Link (S8 -2) --> S3 (23/30):
    Combination 0:  --> pval = 0.12500 / val = 0.293
    Non-significance detected.

    Link (S8 -3) --> S3 (24/30):
    Combination 0:  --> pval = 0.33200 / val = 0.194
    Non-significance detected.

    Link (S9 -1) --> S3 (25/30):
    Combination 0:  --> pval = 0.03000 / val = 0.384
    No conditions of dimension 0 left.

    Link (S9 -2) --> S3 (26/30):
    Combination 0:  --> pval = 0.24400 / val = 0.243
    Non-significance detected.

    Link (S9 -3) --> S3 (27/30):
    Combination 0:  --> pval = 0.11600 / val = 0.300
    Non-significance detected.

    Link (S10 -1) --> S3 (28/30):
    Combination 0:  --> pval = 0.45000 / val = 0.161
    Non-significance detected.

    Link (S10 -2) --> S3 (29/30):
    Combination 0:  --> pval = 0.19000

    Combination 0: (S5 -3)  --> pval = 0.02500 / val = 0.279
    No conditions of dimension 1 left.

    Link (S5 -2) --> S4 (8/18):
    Combination 0: (S5 -3)  --> pval = 0.05200 / val = 0.279
    Non-significance detected.

    Link (S8 -1) --> S4 (9/18):
    Combination 0: (S5 -3)  --> pval = 0.41100 / val = 0.169
    Non-significance detected.

    Link (S10 -2) --> S4 (10/18):
    Combination 0: (S5 -3)  --> pval = 0.15800 / val = 0.205
    Non-significance detected.

    Link (S3 -2) --> S4 (11/18):
    Combination 0: (S5 -3)  --> pval = 0.03600 / val = 0.265
    No conditions of dimension 1 left.

    Link (S7 -3) --> S4 (12/18):
    Combination 0: (S5 -3)  --> pval = 0.82500 / val = 0.070
    Non-significance detected.

    Link (S2 -3) --> S4 (13/18):
    Combination 0: (S5 -3)  --> pval = 0.88700 / val = 0.037
    Non-significance detected.

    Link (S10 -3) --> S4 (14/18):
    Combination 0: (S5 -3)  --> pval = 0.60100 / val = 0.103
    Non-significance detected.

    Link 

    Combination 0: (S9 -2)  --> pval = 0.00300 / val = 0.423
    No conditions of dimension 1 left.

    Link (S10 -1) --> S5 (13/22):
    Combination 0: (S9 -2)  --> pval = 0.14200 / val = 0.256
    Non-significance detected.

    Link (S8 -1) --> S5 (14/22):
    Combination 0: (S9 -2)  --> pval = 0.00300 / val = 0.404
    No conditions of dimension 1 left.

    Link (S2 -2) --> S5 (15/22):
    Combination 0: (S9 -2)  --> pval = 0.04500 / val = 0.269
    No conditions of dimension 1 left.

    Link (S8 -2) --> S5 (16/22):
    Combination 0: (S9 -2)  --> pval = 0.01400 / val = 0.340
    No conditions of dimension 1 left.

    Link (S7 -3) --> S5 (17/22):
    Combination 0: (S9 -2)  --> pval = 0.00800 / val = 0.299
    No conditions of dimension 1 left.

    Link (S10 -3) --> S5 (18/22):
    Combination 0: (S9 -2)  --> pval = 0.01500 / val = 0.334
    No conditions of dimension 1 left.

    Link (S2 -3) --> S5 (19/22):
    Combination 0: (S9 -2)  --> pval = 0.01500 / val = 0.343
    No 

...

        pval = 0.34500 | val = 0.157

        link (S10 -3) --> S3 (39/39):
        with conds_y = [ (S5 -2) ]
        with conds_x = [ (S9 -4) (S1 -6) (S5 -5) ]
        pval = 0.78400 | val = 0.107

        link (S1 0) --> S4 (1/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ ]
        pval = 0.53800 | val = 0.146

        link (S1 -1) --> S4 (2/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ ]
        pval = 0.96900 | val = 0.036

        link (S1 -2) --> S4 (3/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ ]
        pval = 0.01700 | val = 0.354

        link (S1 -3) --> S4 (4/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ ]
        pval = 0.25700 | val = 0.243

        link (S2 0) --> S4 (5/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ (S4 -1) ]
        pval = 0.97200 | val = 0.031

        link (S2 -1) --> S4 (6/39):
        with conds_y = [ (S4 -1) ]
        with conds_x = [ (S4 -2) ]
        

        pval = 0.11700 | val = 0.302

        link (S4 -3) --> S5 (16/39):
        with conds_y = [ (S9 -2) (S4 -1) (S5 -2) ]
        with conds_x = [ (S4 -4) ]
        pval = 0.01500 | val = 0.424

        link (S5 -1) --> S5 (17/39):
        with conds_y = [ (S9 -2) (S4 -1) (S5 -2) ]
        with conds_x = [ (S9 -3) (S4 -2) (S5 -3) ]
        pval = 0.76600 | val = 0.279

        link (S5 -2) --> S5 (18/39):
        with conds_y = [ (S9 -2) (S4 -1) ]
        with conds_x = [ (S9 -4) (S4 -3) (S5 -4) ]
        pval = 0.05700 | val = 0.284

        link (S5 -3) --> S5 (19/39):
        with conds_y = [ (S9 -2) (S4 -1) (S5 -2) ]
        with conds_x = [ (S9 -5) (S4 -4) (S5 -5) ]
        pval = 0.72100 | val = 0.203

        link (S6 0) --> S5 (20/39):
        with conds_y = [ (S9 -2) (S4 -1) (S5 -2) ]
        with conds_x = [ ]
        pval = 0.10200 | val = 0.177

        link (S6 -1) --> S5 (21/39):
        with conds_y = [ (S9 -2) (S4 -1) (S5 -2) ]
        with conds_x = [ ]
        pva

        pval = 0.08000 | val = 0.191

        link (S9 -1) --> S6 (33/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -2) (S5 -3) ]
        pval = 0.87200 | val = 0.045

        link (S9 -2) --> S6 (34/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -3) (S5 -4) ]
        pval = 0.91000 | val = 0.026

        link (S9 -3) --> S6 (35/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -4) (S5 -5) ]
        pval = 0.53700 | val = 0.129

        link (S10 0) --> S6 (36/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -1) (S1 -3) (S5 -2) ]
        pval = 0.48000 | val = 0.091

        link (S10 -1) --> S6 (37/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -2) (S1 -4) (S5 -3) ]
        pval = 0.26700 | val = 0.155

        link (S10 -2) --> S6 (38/39):
        with conds_y = [ ]
        with conds_x = [ (S9 -3) (S1 -5) (S5 -4) ]
        pval = 0.60900 | val = 0.114

        link (S10 -3) --> S6 (39/39):
        with conds_y = [ ]
 

        pval = 0.91200 | val = 0.082

        link (S4 -1) --> S8 (14/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S4 -2) ]
        pval = 0.08900 | val = 0.246

        link (S4 -2) --> S8 (15/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S4 -3) ]
        pval = 0.12000 | val = 0.243

        link (S4 -3) --> S8 (16/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S4 -4) ]
        pval = 0.31000 | val = 0.168

        link (S5 0) --> S8 (17/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S9 -2) (S4 -1) (S5 -2) ]
        pval = 0.36600 | val = 0.181

        link (S5 -1) --> S8 (18/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S9 -3) (S4 -2) (S5 -3) ]
        pval = 0.01900 | val = 0.315

        link (S5 -2) --> S8 (19/39):
        with conds_y = [ (S9 -3) ]
        with conds_x = [ (S9 -4) (S4 -3) (S5 -4) ]
        pval = 0.88500 | val = 0.094

        link (S5 -3) --> S8 (20/39):
        

        pval = 0.18400 | val = 0.228

        link (S8 -1) --> S9 (30/39):
        with conds_y = [ (S9 -1) (S5 -2) ]
        with conds_x = [ (S9 -4) ]
        pval = 0.42900 | val = 0.184

        link (S8 -2) --> S9 (31/39):
        with conds_y = [ (S9 -1) (S5 -2) ]
        with conds_x = [ (S9 -5) ]
        pval = 0.34200 | val = 0.136

        link (S8 -3) --> S9 (32/39):
        with conds_y = [ (S9 -1) (S5 -2) ]
        with conds_x = [ (S9 -6) ]
        pval = 0.76800 | val = 0.078

        link (S9 -1) --> S9 (33/39):
        with conds_y = [ (S5 -2) ]
        with conds_x = [ (S9 -2) (S5 -3) ]
        pval = 0.00300 | val = 0.426

        link (S9 -2) --> S9 (34/39):
        with conds_y = [ (S9 -1) (S5 -2) ]
        with conds_x = [ (S9 -3) (S5 -4) ]
        pval = 0.11500 | val = 0.172

        link (S9 -3) --> S9 (35/39):
        with conds_y = [ (S9 -1) (S5 -2) ]
        with conds_x = [ (S9 -4) (S5 -5) ]
        pval = 0.85100 | val = 0.090

        link (S10 0) --> S9 

In [ ]:
my_data=np.genfromtxt(file_dir+'/Gadus_morhua_PCMCI_prep.csv',delimiter=",")
my_data=np.nan_to_num(my_data)
#print(my_data)
# Initialize dataframe object, specify time axis and variable names
var_names = ["S1","S2","S3","S4","S5","S6","S7","S8","S9","S10"]
dataframe = pp.DataFrame(my_data, 
                         datatime = np.arange(len(my_data)), 
                         var_names=var_names)

In [ ]:
parcorr = ParCorr(significance='shuffle_test',confidence="bootstrap")
pcmci = PCMCI(
    dataframe=dataframe, 
    cond_ind_test=parcorr,
    verbosity=1)

In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
%%capture cap --no-stderr
#Run and save analysis output

pcmci.verbosity = 1
results = pcmci.run_pcmci(tau_max=3, pc_alpha=None)

with open(file_dir+"/Seascapes/Gadus_morhua_PCMCI.txt","w") as f:
    f.write(cap.stdout)
    f.close()

In [ ]:
%%capture cap --no-stderr
q_matrix = pcmci.get_corrected_pvalues(p_matrix=results['p_matrix'], fdr_method='fdr_bh')
pcmci.print_significant_links(
        p_matrix = results['p_matrix'], 
        q_matrix = q_matrix,
        val_matrix = results['val_matrix'],
        conf_matrix=results["conf_matrix"],
        alpha_level = 0.01)
with open(file_dir+"/Seascapes/Gadus_morhua_PCMCI.txt","a") as f:
    f.write(cap.stdout)
    f.close()

In [ ]:
print(type(results["p_matrix"]))
pm=results["p_matrix"]
ci=results["conf_matrix"]
uci=ci[:,:,:,1]
lci=ci[:,:,:,0]
ciw=uci-lci
val=results["val_matrix"]
#print(ciw)

# Write the array to disk
with open("C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Seascapes/PCMCI_p_matrix.csv", 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(pm.shape))
    print(pm.shape)
    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in pm:
    
        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%1.10f')

        # Writing out a break to indicate different slices...
        outfile.write('\n')
        
with open("C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Seascapes/PCMCI_uci_matrix.csv", 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(uci.shape))
    print(uci.shape)
    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in uci:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%1.10f')

        # Writing out a break to indicate different slices...
        outfile.write('\n')
        
with open("C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Seascapes/PCMCI_lci_matrix.csv", 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(lci.shape))
    print(lci.shape)
    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in lci:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%1.10f')

        # Writing out a break to indicate different slices...
        outfile.write('\n')
        
with open("C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Seascapes/PCMCI_ciw_matrix.csv", 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(ciw.shape))
    print(ciw.shape)
    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in ciw:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%1.10f')

        # Writing out a break to indicate different slices...
        outfile.write('\n')
        
with open("C:/Users/mrademaker/Documents/Research projects/STCNWS/DATRAS/cpue_length_hour/Seascapes/PCMCI_val_matrix.csv", 'w') as outfile:
    # I'm writing a header here just for the sake of readability
    # Any line starting with "#" will be ignored by numpy.loadtxt
    outfile.write('# Array shape: {0}\n'.format(val.shape))
    print(val.shape)
    # Iterating through a ndimensional array produces slices along
    # the last axis. This is equivalent to data[i,:,:] in this case
    for data_slice in val:

        # The formatting string indicates that I'm writing out
        # the values in left-justified columns 7 characters in width
        # with 2 decimal places.  
        np.savetxt(outfile, data_slice, fmt='%1.10f')

        # Writing out a break to indicate different slices...
        outfile.write('\n')

In [ ]:
link_matrix = pcmci.return_significant_parents(pq_matrix=q_matrix,
                        val_matrix=results['val_matrix'], alpha_level=0.01)['link_matrix']

In [ ]:
tp.plot_graph(
    val_matrix=results['val_matrix'],
    link_matrix=link_matrix,
    var_names=var_names,
    link_colorbar_label='cross-MCI',
    node_colorbar_label='auto-MCI',
    )